In [ ]:
first = pd.read_csv('Zenodo Inconel Single/imageDatasingle.csv', sep = ';')
second = pd.read_csv('/Zenodo Inconel Single/trackDatasingle.csv', sep = ';')
third = pd.merge(first, second, on = 't', how = 'inner')
filtered_df = df[df['fileName'] == specific_string]
whatyoumightwanttokeep = ['t','Xpos','Ypos','Vnom','Pnom']
source_folder = '/content/sample_data/extracted8/Images'

# Step 2: List all .tif files in the source folder
all_files = [f for f in os.listdir(source_folder) if f.lower().endswith('.jpeg')]

# Custom sort key function for JPEG files in the format fr15457985_ts833846284597_fe8.jpeg
def sort_key(filename):
    parts = filename.split('_')
    fr_part = int(parts[0])
    ts_part = int(parts[1])
    fe_part = int(parts[2].split('.')[0])
    return fe_part

# Step 3: Sort the files
all_files.sort(key = sort_key)

total_files = len(all_files)
print(f"Total .tif files found: {total_files}")
print(all_files)
count = 0
raw_image = []
processed_image = []
time = []
Pos = []
for filename in all_files:
    file_path = os.path.join(source_folder, filename)
    temp = third[third['fileName'] == filename]
    if temp.empty !=  True and count<1024:
      with Image.open(file_path) as img:
          image_array = np.array(img)
          raw_image.append(image_array)
          processed_image.append(np.mean(image_array, axis=2).astype(np.uint8))
          time.append(temp['t'].values[0])
          Pos.append(temp[['Xpos','Ypos']].values[0])
    count = (count+1) % (1024*2)

raw_image = np.array(raw_image)
processed_image = np.array(processed_image)
Pos = np.array(Pos)
time = np.array(time)
print(raw_image.shape)
print(processed_image.shape)
print(Pos.shape)
print(time.shape)

In [ ]:
# Define the path to the .dat file
file_path = '/dataverse_files (2).zip (Unzipped Files)/bichromatic/LMDP.dat'

# Load the .dat file using pandas
data = pd.read_csv(file_path,
                   delimiter=',',  # CSV file
                   quotechar='"',  # Handle quoted strings
                   skipinitialspace=True,  # Handle spaces after delimiters
                   engine='python')  # Use the 'python' engine to handle more complex quoting

# Display the first few rows of the dataframe
print(data.head())

# Display the column names to verify
print(data.column)

# Define paths
dat_file_path = '/bichromatic/LMDP.dat'
destination_hdf5_file = '/MeltPoolViT/Final_DatasetV2.h5'

# Load the .dat file using pandas
data = pd.read_csv(dat_file_path,
                   delimiter=',',
                   quotechar='"',
                   skipinitialspace=True,
                   engine='python')

# Sort by 'ImageId'
data = data.sort_values(by='ImageId')

# Drop unnecessary columns
data = data.drop(columns=['DateTime', 'Gain', 'FPS', 'ImageId'])

# Process 'Timestamp' to start at 0 (subtract the minimum value)
data['Timestamp'] = data['Timestamp'] - data['Timestamp'].min()

# Extract relevant columns
timestamps = data['Timestamp'].values
exposure_times = data['ExposureTime'].values
print(timestamps)
print(exposure_times)

In [ ]:
with h5py.File(destination_hdf5_file, 'a') as hdf:
    # Access or create the group and subgroup
    group = hdf.require_group('source1')
    subgroup = group.require_group('WAAM')


    subgroup.create_dataset('Timestamp', data=timestamps, dtype='float64')

    subgroup.create_dataset('ExposureTime', data=exposure_times, dtype='float64')

    print(f"Data successfully saved to {destination_hdf5_file}")

# Optional: Display the processed data
print(data.head())